In [1]:
!pip install -U sentence-transformers

     -------------------------------------- 86.0/86.0 kB 440.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125960 sha256=09e927e627539f5a7de90ab1170fdd1f850bed361186f2d688eb3888367b6631
  Stored in directory: c:\users\dan\appdata\local\pip\cache\wheels\62\f2\10\1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
!pip install -U text2vec

     -------------------------------------- 70.8/70.8 kB 203.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 60.0/60.0 kB 264.4 kB/s eta 0:00:00
  Created wheel for text2vec: filename=text2vec-1.1.8-py3-none-any.whl size=56558 sha256=e427d1dce52380178f39dc11ca328f5ef6529d5beb30ecbbcb21726967d84bcd
  Stored in directory: c:\users\dan\appdata\local\pip\cache\wheels\85\27\60\6dc176b368eb5f1b78909026e8a3a6e6aa98d5a40a5b077676
Successfully built text2vec


## sentence BERT

In [1]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("shibing624/text2vec-base-chinese")

model = AutoModel.from_pretrained("shibing624/text2vec-base-chinese")

c:\Users\Dan\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
with open('fubon.json',"r",encoding="UTF-8") as f:
    corpus_data = json.load(f)

documents=dict()
for line in corpus_data:
    documents.update({list(line.keys())[0]:list(line.values())[0]})
corpus = [list(data.keys())[0] for data in corpus_data]
print(corpus[0])

富邦人壽新平準終身壽險_老年住院醫療提前給付保險金的給付


In [3]:
import numpy as np
import torch
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
from tqdm import tqdm
vecs = []
for sentence in tqdm(corpus):
    tokens = tokenizer(sentence, max_length = 256, padding='max_length', truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model(**tokens)[0].cpu().numpy()
        vecs.append(output.reshape(-1))

vecs = np.vstack(vecs)

# 打印dense embedding的形状
print(f"Shape of dense embeddings: {vecs.shape}")
np.save('fubon_data_sentencebert.npy', vecs)

100%|██████████| 1907/1907 [00:25<00:00, 75.11it/s]


Shape of dense embeddings: (1907, 196608)


In [4]:
import numpy as np 
vecs = np.load("fubon_data_sentencebert.npy")

In [5]:
import faiss
import numpy as np
import torch
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("shibing624/text2vec-base-chinese")

model = AutoModel.from_pretrained("shibing624/text2vec-base-chinese").to(device)

### query text here !
query_text = "身故保險金是什麼?"

print(query_text)
# 將要比較的句子轉換成dense embedding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokens = tokenizer(query_text, max_length = 256, padding='max_length', truncation=True, return_tensors="pt").to(device)
with torch.no_grad():
    query_vec = model(**tokens)[0].cpu().numpy().reshape(-1)

# 計算query與向量庫中所有向量的相似度
index = faiss.IndexFlatL2(vecs.shape[1])
index.add(vecs)
distances, indices = index.search(np.array([query_vec]), k=5)

# 打印出相似度最高的前5筆資料
for i, index in enumerate(indices[0]):
    print(f"Rank {i+1}: {corpus[index]}, similarity score: {distances[0][i]}")
    print(f"article : \n {documents.get(corpus[index])}")

身故保險金是什麼?
Rank 1: 富邦人壽安心理財人生傷害暨健康一年定期保險_保險範圍：身故保險金或喪葬費用保險金的給付, similarity score: 66209.90625
article : 
 被保險人於本契約有效期間內身故者，本公司按要保人投保計畫別對照附表一所載之金額給付身故保險金或喪葬費用保險金後，本契約效力即行終止。訂立本契約時，以受監護宣告尚未撤銷者為被保險人，其身故保險金變更為喪葬費用保險金。前項被保險人於民國九十九年二月三日（含）以後所投保之喪葬費用保險金額總和（不限本公司），不得超過遺產及贈與稅法第十七條有關遺產稅喪葬費扣除額之半數，其超過部分本公司不負給付責任，本公司並應無息退還該超過部分之已繳保險費。前項情形，如要保人向二家（含）以上保險公司投保，或向同一保險公司投保數個保險契（附）約，且其投保之喪葬費用保險金額合計超過前項所定之限額者，本公司於所承保之喪葬費用金額範圍內，依各要保書所載之要保時間先後，依約給付喪葬費用保險金至前項喪葬費用額度上限為止，如有二家以上保險公司之保險契約要保時間相同或無法區分其要保時間之先後者，各該保險公司應依其喪葬費用保險金額與扣除要保時間在先之保險公司應理賠之金額後所餘之限額比例分擔其責任。
Rank 2: 富邦人壽願景人生傷害暨健康一年定期保險_保險範圍：身故保險金或喪葬費用保險金的給付, similarity score: 68176.359375
article : 
 被保險人於本契約有效期間內身故者，本公司按要保人投保計畫別對照附表一所載之金額給付身故保險金或喪葬費用保險金後，本契約效力即行終止。訂立本契約時，以受監護宣告尚未撤銷者為被保險人，其身故保險金變更為喪葬費用保險金。前項被保險人於民國九十九年二月三日（含）以後所投保之喪葬費用保險金額總和（不限本公司），不得超過遺產及贈與稅法第十七條有關遺產稅喪葬費扣除額之半數，其超過部分本公司不負給付責任，本公司並應無息退還該超過部分之已繳保險費。前項情形，如要保人向二家（含）以上保險公司投保，或向同一保險公司投保數個保險契（附）約，且其投保之喪葬費用保險金額合計超過前項所定之限額者，本公司於所承保之喪葬費用金額範圍內，依各要保書所載之要保時間先後，依約給付喪葬費用保險金至前項喪葬費用額度上限為止，如有二家以上保險公司之保險契約要保時間相同或無